In [2]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope
import statsmodels.api as sm
from itertools import combinations
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import warnings
import logging
from datetime import datetime

In [3]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'multivariate_analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

In [4]:
def clean_column_names(*columns):
    """Helper function to clean multiple column names"""
    try:
        return [str(col).strip().lower().replace("'", "").replace('"', "") for col in columns]
    except Exception as e:
        logger.error(f"Error cleaning column names: {str(e)}")
        raise

def print_formatted_results(result):
    """Helper function to format and print results"""
    try:
        if isinstance(result, dict):
            for key, value in result.items():
                print(f"\n{key.replace('_', ' ').title()}:")
                if isinstance(value, dict):
                    for subkey, subvalue in value.items():
                        if isinstance(subvalue, dict):
                            print(f"  {subkey.replace('_', ' ').title()}:")
                            for subsubkey, subsubvalue in subvalue.items():
                                print(f"    {subsubkey.replace('_', ' ').title()}: {subsubvalue}")
                        else:
                            print(f"  {subkey.replace('_', ' ').title()}: {subvalue}")
                else:
                    print(f"  {value}")
        else:
            print(result)
    except Exception as e:
        logger.error(f"Error formatting results: {str(e)}")
        print(f"Error formatting results: {str(e)}")

def run_multivariate_queries(multivariate_analyzer, rag_system, agent, data):
    """Run example queries with improved handling"""
    queries = [
        {
            "query": "Analyze the relationships between age, bmi, and charges",
            "expected_tool": "Multivariate Statistics",
            "backup_func": lambda: multivariate_analyzer.compute_multivariate_stats(['age', 'bmi', 'charges'])
        },
        {
            "query": "Perform PCA on the numeric variables age, bmi, and charges",
            "expected_tool": "PCA Analysis",
            "backup_func": lambda: multivariate_analyzer.perform_pca(['age', 'bmi', 'charges'])
        },
        {
            "query": "How do the relationships between age, bmi, and charges vary by smoking status?",
            "expected_tool": "Categorical Group Analysis",
            "backup_func": lambda: multivariate_analyzer.analyze_by_categorical(['age', 'bmi', 'charges'], 'smoker')
        },
        {
            "query": "What patterns emerge in the relationships between demographic and health variables?",
            "expected_tool": "RAG Query",
            "backup_func": lambda: rag_system.query_rag("What are the key patterns in relationships between demographic and health variables in insurance pricing?")
        }
    ]

    for i, query_info in enumerate(queries, 1):
        print(f"\nQuery {i}: {query_info['query']}")
        print("-" * 80)
        
        try:
            logger.info(f"Processing query {i}: {query_info['query']}")
            result = agent.run(query_info['query'])
            print("\nAnalysis Result (via Agent):")
            print_formatted_results(result)
            
        except Exception as e:
            logger.warning(f"Agent analysis failed for query {i}, falling back to direct analysis")
            print(f"\nFalling back to direct analysis...")
            try:
                result = query_info['backup_func']()
                print("\nAnalysis Result (via Backup):")
                print_formatted_results(result)
            except Exception as backup_error:
                logger.error(f"Backup analysis failed for query {i}: {str(backup_error)}")
                print(f"Analysis failed: {str(backup_error)}")
            
        print("\n" + "=" * 80)

def run_additional_insights(rag_system):
    """Run additional RAG queries for deeper insights"""
    insight_queries = [
        "How do multiple health indicators combine to influence insurance charges?",
        "What are the key variable interactions in determining insurance costs?",
        "How do demographic factors modify the relationships between health metrics?",
        "What patterns emerge from the multivariate analysis across different groups?"
    ]

    print("\nGenerating Additional Insights:")
    print("=" * 80)
    
    for i, query in enumerate(insight_queries, 1):
        print(f"\nInsight Query {i}: {query}")
        print("-" * 80)
        try:
            logger.info(f"Processing insight query {i}: {query}")
            result = rag_system.query_rag(query)
            print("\nInsight Result:")
            print(result)
            print("=" * 80)
        except Exception as e:
            logger.error(f"Error generating insight for query {i}: {str(e)}")
            print(f"Error generating insight: {str(e)}")

In [5]:
class MultivariateAnalyzer:
    def __init__(self, data):
        """Initialize with pandas DataFrame"""
        self.data = data
        self.numeric_cols = set(data.select_dtypes(include=[np.number]).columns)
        self.categorical_cols = set(data.select_dtypes(exclude=[np.number]).columns)
        self.scaler = StandardScaler()
        self.pca = None
        logger.info("MultivariateAnalyzer initialized successfully")
        logger.debug(f"Numeric columns: {self.numeric_cols}")
        logger.debug(f"Categorical columns: {self.categorical_cols}")
        
    def validate_columns(self, columns, required_type=None):
        """Validate column existence and type"""
        if not all(col in self.data.columns for col in columns):
            missing = [col for col in columns if col not in self.data.columns]
            raise ValueError(f"Columns not found in dataset: {missing}")
        
        if required_type == 'numeric':
            non_numeric = [col for col in columns if col not in self.numeric_cols]
            if non_numeric:
                raise ValueError(f"Non-numeric columns found: {non_numeric}")
        elif required_type == 'categorical':
            non_cat = [col for col in columns if col not in self.categorical_cols]
            if non_cat:
                raise ValueError(f"Non-categorical columns found: {non_cat}")
    
    def compute_multivariate_stats(self, variables):
        """Compute comprehensive multivariate statistics"""
        try:
            logger.info(f"Computing multivariate statistics for variables: {variables}")
            
            # Clean and validate column names
            variables = clean_column_names(*variables)
            self.validate_columns(variables, 'numeric')
            
            data = self.data[variables].dropna()
            logger.debug(f"Analysis sample size: {len(data)}")
            
            # Correlation matrix
            corr_matrix = data.corr().round(3)
            logger.debug("Correlation matrix computed successfully")
            
            # Partial correlations
            partial_corrs = {}
            if len(variables) > 2:
                for var1, var2 in combinations(variables, 2):
                    try:
                        control_vars = [var for var in variables if var not in [var1, var2]]
                        partial_corrs[f"{var1}-{var2}"] = {
                            'correlation': f"{self._compute_partial_correlation(var1, var2, control_vars):.3f}",
                            'controlling_for': ', '.join(control_vars)
                        }
                    except Exception as e:
                        logger.warning(f"Error computing partial correlation for {var1}-{var2}: {str(e)}")
                        partial_corrs[f"{var1}-{var2}"] = f"Error: {str(e)}"
            
            # Multivariate normality test
            normality_results = self._test_multivariate_normality(data)
            
            # Outlier detection
            outliers = self._detect_multivariate_outliers(data)
            
            # Interpret relationships
            relationships = self._interpret_relationships(corr_matrix)
            
            result = {
                'summary': {
                    'variables_analyzed': len(variables),
                    'sample_size': len(data),
                    'missing_values': self.data[variables].isna().sum().to_dict()
                },
                'correlation_matrix': corr_matrix.to_dict(),
                'partial_correlations': partial_corrs,
                'multivariate_normality': normality_results,
                'outliers': outliers,
                'relationships': relationships
            }
            
            logger.info("Multivariate statistics computation completed successfully")
            return result
            
        except Exception as e:
            logger.error(f"Error in multivariate analysis: {str(e)}")
            return f"Error in multivariate analysis: {str(e)}"
    
    def _compute_partial_correlation(self, var1, var2, control_vars):
        """Compute partial correlation between two variables"""
        try:
            data = self.data[[var1, var2] + control_vars].dropna()
            
            # Prepare data for partial correlation
            X = sm.add_constant(data[control_vars])
            
            # Residuals for var1
            model1 = sm.OLS(data[var1], X).fit()
            resid1 = model1.resid
            
            # Residuals for var2
            model2 = sm.OLS(data[var2], X).fit()
            resid2 = model2.resid
            
            # Correlation between residuals
            partial_corr = stats.pearsonr(resid1, resid2)[0]
            
            return partial_corr
        except Exception as e:
            logger.error(f"Error computing partial correlation: {str(e)}")
            raise
    
    def _test_multivariate_normality(self, data):
        """Test for multivariate normality using Mardia's test"""
        try:
            # Standardize the data
            scaled_data = self.scaler.fit_transform(data)
            
            # Calculate Mardia's coefficients
            n_samples = len(scaled_data)
            
            # Skewness
            m_skew = self._mardias_skewness(scaled_data)
            skew_stat = (n_samples / 6) * m_skew
            skew_df = data.shape[1] * (data.shape[1] + 1) * (data.shape[1] + 2) / 6
            skew_p_value = 1 - stats.chi2.cdf(skew_stat, skew_df)
            
            # Kurtosis
            m_kurt = self._mardias_kurtosis(scaled_data)
            kurt_stat = (m_kurt - data.shape[1] * (data.shape[1] + 2)) / np.sqrt(8 * data.shape[1] * (data.shape[1] + 2) / n_samples)
            kurt_p_value = 2 * (1 - stats.norm.cdf(abs(kurt_stat)))
            
            return {
                'skewness': {
                    'statistic': f"{skew_stat:.3f}",
                    'p_value': f"{skew_p_value:.3f}",
                    'interpretation': 'Normal' if skew_p_value > 0.05 else 'Non-normal'
                },
                'kurtosis': {
                    'statistic': f"{kurt_stat:.3f}",
                    'p_value': f"{kurt_p_value:.3f}",
                    'interpretation': 'Normal' if kurt_p_value > 0.05 else 'Non-normal'
                }
            }
        except Exception as e:
            logger.error(f"Error in multivariate normality test: {str(e)}")
            raise

    def _mardias_skewness(self, data):
        """Calculate Mardia's skewness coefficient"""
        try:
            n_samples = len(data)
            centered_data = data - np.mean(data, axis=0)
            cov = np.cov(data.T)
            cov_inv = np.linalg.inv(cov)
            
            m_skew = 0
            for i in range(n_samples):
                for j in range(n_samples):
                    m_skew += np.power(
                        np.dot(np.dot(centered_data[i], cov_inv), centered_data[j].T),
                        3
                    )
            
            return m_skew / (n_samples * n_samples)
        except Exception as e:
            logger.error(f"Error calculating Mardia's skewness: {str(e)}")
            raise
    
    def _mardias_kurtosis(self, data):
        """Calculate Mardia's kurtosis coefficient"""
        try:
            n_samples = len(data)
            centered_data = data - np.mean(data, axis=0)
            cov = np.cov(data.T)
            cov_inv = np.linalg.inv(cov)
            
            m_kurt = 0
            for i in range(n_samples):
                m_kurt += np.power(
                    np.dot(np.dot(centered_data[i], cov_inv), centered_data[i].T),
                    2
                )
                
            return m_kurt / n_samples
        except Exception as e:
            logger.error(f"Error calculating Mardia's kurtosis: {str(e)}")
            raise
    
    def _detect_multivariate_outliers(self, data):
        """Detect multivariate outliers using Mahalanobis distance"""
        try:
            outlier_detector = EllipticEnvelope(contamination=0.1, random_state=42)
            outlier_labels = outlier_detector.fit_predict(data)
            
            outlier_indices = np.where(outlier_labels == -1)[0]
            total_outliers = len(outlier_indices)
            
            return {
                'num_outliers': total_outliers,
                'percentage_outliers': f"{(total_outliers / len(data)) * 100:.2f}%",
                'outlier_indices': outlier_indices.tolist()
            }
        except Exception as e:
            logger.error(f"Error detecting multivariate outliers: {str(e)}")
            raise
    
    def _interpret_relationships(self, corr_matrix):
        """Interpret relationships between variables"""
        try:
            interpretations = {}
            for var1 in corr_matrix.index:
                for var2 in corr_matrix.columns:
                    if var1 < var2:  # Avoid duplicates and self-correlations
                        corr = corr_matrix.loc[var1, var2]
                        interpretations[f"{var1}-{var2}"] = {
                            'correlation': f"{corr:.3f}",
                            'strength': 'Strong' if abs(corr) > 0.7 else 'Moderate' if abs(corr) > 0.3 else 'Weak',
                            'direction': 'Positive' if corr > 0 else 'Negative'
                        }
            return interpretations
        except Exception as e:
            logger.error(f"Error interpreting relationships: {str(e)}")
            raise

    def _interpret_group_differences(self, group_stats):
        """Interpret differences between groups with detailed analysis"""
        try:
            interpretations = {}
            
            # Extract variables analyzed
            sample_analysis = next(iter(group_stats.values()))['analysis']
            if isinstance(sample_analysis, dict) and 'correlation_matrix' in sample_analysis:
                variables = list(sample_analysis['correlation_matrix'].keys())
            else:
                variables = []
                
            for category, stats in group_stats.items():
                if isinstance(stats['analysis'], dict):
                    # Basic group characteristics
                    interpretations[category] = {
                        'sample_characteristics': {
                            'size': stats['sample_size'],
                            'proportion': stats['percentage']
                        }
                    }
                    
                    # Extract key findings
                    key_findings = []
                    
                    # Check correlation patterns
                    if 'correlation_matrix' in stats['analysis']:
                        corr_matrix = stats['analysis']['correlation_matrix']
                        strong_correlations = []
                        for var1 in variables:
                            for var2 in variables:
                                if var1 < var2:  # Avoid duplicates
                                    corr = corr_matrix.get(var1, {}).get(var2, 0)
                                    if abs(corr) > 0.7:
                                        strong_correlations.append({
                                            'variables': f"{var1}-{var2}",
                                            'correlation': f"{corr:.3f}",
                                            'strength': 'Strong positive' if corr > 0 else 'Strong negative'
                                        })
                        
                        if strong_correlations:
                            key_findings.extend([f"Strong {corr['strength'].lower()} correlation ({corr['correlation']}) between {corr['variables']}"
                                              for corr in strong_correlations])
                    
                    # Check multivariate normality
                    if 'multivariate_normality' in stats['analysis']:
                        norm_results = stats['analysis']['multivariate_normality']
                        if norm_results['skewness']['interpretation'] == 'Non-normal' or \
                           norm_results['kurtosis']['interpretation'] == 'Non-normal':
                            key_findings.append("Data shows significant departure from multivariate normality")
                    
                    # Check outliers
                    if 'outliers' in stats['analysis']:
                        outlier_info = stats['analysis']['outliers']
                        if float(outlier_info['percentage_outliers'].rstrip('%')) > 5:
                            key_findings.append(f"High proportion of outliers detected ({outlier_info['percentage_outliers']})")
                    
                    # Add findings to interpretations
                    interpretations[category]['key_findings'] = key_findings if key_findings else ["No significant patterns detected"]
                    
                    # Add comparative analysis
                    if len(interpretations) > 1:
                        other_categories = [c for c in group_stats.keys() if c != category]
                        comparative_insights = self._compare_group_patterns(
                            stats['analysis'],
                            {c: group_stats[c]['analysis'] for c in other_categories}
                        )
                        interpretations[category]['comparative_insights'] = comparative_insights
            
            return interpretations
            
        except Exception as e:
            logger.error(f"Error interpreting group differences: {str(e)}")
            raise

    def _compare_group_patterns(self, current_analysis, other_analyses):
        """Compare patterns between current group and other groups"""
        try:
            insights = []
            
            # Compare correlation patterns
            if isinstance(current_analysis, dict) and 'correlation_matrix' in current_analysis:
                current_corr = current_analysis['correlation_matrix']
                
                for other_group, other_analysis in other_analyses.items():
                    if isinstance(other_analysis, dict) and 'correlation_matrix' in other_analysis:
                        other_corr = other_analysis['correlation_matrix']
                        
                        # Compare correlations
                        for var1 in current_corr.keys():
                            for var2 in current_corr[var1].keys():
                                if var1 < var2:  # Avoid duplicates
                                    curr_corr = current_corr[var1][var2]
                                    other_corr_val = other_corr[var1][var2]
                                    
                                    # Check for substantial differences in correlations
                                    if abs(curr_corr - other_corr_val) > 0.3:
                                        insights.append(
                                            f"Correlation between {var1}-{var2} differs substantially "
                                            f"from {other_group} group "
                                            f"({curr_corr:.3f} vs {other_corr_val:.3f})"
                                        )
            
            return insights if insights else ["No substantial differences in patterns compared to other groups"]
            
        except Exception as e:
            logger.error(f"Error comparing group patterns: {str(e)}")
            raise

    def perform_pca(self, variables):
        """Perform Principal Component Analysis"""
        try:
            logger.info(f"Performing PCA on variables: {variables}")
            
            # Clean and validate
            variables = clean_column_names(*variables)
            self.validate_columns(variables, 'numeric')
            
            data = self.data[variables].dropna()
            logger.debug(f"PCA sample size: {len(data)}")
            
            # Standardize data
            scaled_data = self.scaler.fit_transform(data)
            
            # Perform PCA
            self.pca = PCA()
            pca_result = self.pca.fit_transform(scaled_data)
            
            # Calculate metrics
            explained_variance_ratio = self.pca.explained_variance_ratio_
            cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
            
            # Get component loadings
            loadings = self.pca.components_
            loadings_df = pd.DataFrame(
                loadings.T,
                columns=[f'PC{i+1}' for i in range(len(variables))],
                index=variables
            )
            
            # Determine optimal components
            components_needed = {
                'explained_variance_70': np.argmax(cumulative_variance_ratio >= 0.7) + 1,
                'explained_variance_80': np.argmax(cumulative_variance_ratio >= 0.8) + 1,
                'explained_variance_90': np.argmax(cumulative_variance_ratio >= 0.9) + 1
            }
            
            # Store PCA scores
            pca_scores = pd.DataFrame(
                pca_result,
                columns=[f'PC{i+1}' for i in range(len(variables))]
            )
            
            result = {
                'summary': {
                    'n_components': len(variables),
                    'total_variance_explained': f"{cumulative_variance_ratio[-1]*100:.2f}%",
                    'components_needed': components_needed,
                    'sample_size': len(data)
                },
                'explained_variance': {
                    f'PC{i+1}': f"{var*100:.2f}%" 
                    for i, var in enumerate(explained_variance_ratio)
                },
                'cumulative_variance': {
                    f'PC{i+1}': f"{var*100:.2f}%" 
                    for i, var in enumerate(cumulative_variance_ratio)
                },
                'loadings': loadings_df.round(3).to_dict(),
                'scores': pca_scores.head().to_dict(),  # First few scores for example
                'interpretation': self._interpret_pca_results(
                    loadings_df, 
                    explained_variance_ratio
                )
            }
            
            logger.info("PCA completed successfully")
            return result
            
        except Exception as e:
            logger.error(f"Error in PCA analysis: {str(e)}")
            return f"Error in PCA analysis: {str(e)}"
    
    def _interpret_pca_results(self, loadings_df, explained_variance_ratio):
        """Interpret PCA results with meaningful insights"""
        try:
            interpretations = {}
            
            for i, variance in enumerate(explained_variance_ratio):
                pc_name = f'PC{i+1}'
                loadings = loadings_df[pc_name].abs()
                top_variables = loadings.nlargest(3)
                
                # Determine the loading pattern
                loading_pattern = self._describe_loading_pattern(loadings_df[pc_name])
                
                # Calculate complexity of the component
                complexity = sum(loadings > 0.3)
                
                interpretations[pc_name] = {
                    'variance_explained': f"{variance*100:.2f}%",
                    'dominant_variables': top_variables.index.tolist(),
                    'loading_strengths': {var: f"{val:.3f}" for var, val in top_variables.items()},
                    'importance': 'High' if variance > 0.3 else 'Medium' if variance > 0.1 else 'Low',
                    'complexity': f"{complexity} significant loadings",
                    'pattern_description': loading_pattern['interpretation'],
                    'suggested_name': self._suggest_component_name(loading_pattern, top_variables)
                }
            
            return interpretations
            
        except Exception as e:
            logger.error(f"Error interpreting PCA results: {str(e)}")
            raise
    
    def _describe_loading_pattern(self, loadings):
        """Describe the pattern of loadings for a principal component"""
        try:
            positive_loads = [var for var, load in loadings.items() if load > 0.3]
            negative_loads = [var for var, load in loadings.items() if load < -0.3]
            
            pattern = {
                'positive_correlations': positive_loads,
                'negative_correlations': negative_loads,
                'interpretation': self._interpret_loading_pattern(positive_loads, negative_loads)
            }
            
            return pattern
        except Exception as e:
            logger.error(f"Error describing loading pattern: {str(e)}")
            raise
    
    def _interpret_loading_pattern(self, positive_loads, negative_loads):
        """Interpret the meaning of loading patterns"""
        try:
            # Basic interpretation
            if not positive_loads and not negative_loads:
                return "No strong loadings identified"
            
            interpretation = []
            
            if positive_loads:
                interpretation.append(f"Positive association with {', '.join(positive_loads)}")
            
            if negative_loads:
                interpretation.append(f"Negative association with {', '.join(negative_loads)}")
            
            # Check for opposing patterns
            if positive_loads and negative_loads:
                interpretation.append("Represents a contrast between variable groups")
            
            return " | ".join(interpretation)
            
        except Exception as e:
            logger.error(f"Error interpreting loading pattern: {str(e)}")
            raise
    
    def _suggest_component_name(self, loading_pattern, top_variables):
        """Suggest a meaningful name for the principal component"""
        try:
            positive_vars = loading_pattern['positive_correlations']
            negative_vars = loading_pattern['negative_correlations']
            
            if not positive_vars and not negative_vars:
                return "Undefined Component"
            
            # Simple case: only positive loadings
            if positive_vars and not negative_vars:
                return f"General {'/'.join(positive_vars[:2])} Factor"
            
            # Simple case: only negative loadings
            if negative_vars and not positive_vars:
                return f"Inverse {'/'.join(negative_vars[:2])} Factor"
            
            # Complex case: both positive and negative
            return f"{'/'.join(positive_vars[:1])} vs {'/'.join(negative_vars[:1])} Contrast"
            
        except Exception as e:
            logger.error(f"Error suggesting component name: {str(e)}")
            raise

    def analyze_by_categorical(self, numeric_vars, categorical_var):
        """Analyze multivariate relationships within categorical groups"""
        try:
            logger.info(f"Analyzing relationships by categorical variable: {categorical_var}")
            
            # Clean and validate column names
            numeric_vars = clean_column_names(*numeric_vars)
            categorical_var = clean_column_names(categorical_var)[0]
            
            # Validate columns
            self.validate_columns(numeric_vars, 'numeric')
            self.validate_columns([categorical_var], 'categorical')
            
            group_stats = {}
            unique_categories = self.data[categorical_var].unique()
            logger.debug(f"Found {len(unique_categories)} unique categories")
            
            for category in unique_categories:
                group_data = self.data[self.data[categorical_var] == category]
                temp_analyzer = MultivariateAnalyzer(group_data)
                group_stats[str(category)] = {
                    'sample_size': len(group_data),
                    'percentage': f"{(len(group_data) / len(self.data)) * 100:.1f}%",
                    'analysis': temp_analyzer.compute_multivariate_stats(numeric_vars)
                }
            
            # Compare groups and interpret differences
            group_comparison = self._compare_groups(numeric_vars, categorical_var)
            interpretation = self._interpret_group_differences(group_stats)
            
            result = {
                'group_statistics': group_stats,
                'group_comparison': group_comparison,
                'interpretation': interpretation,
                'summary': {
                    'total_sample_size': len(self.data),
                    'number_of_groups': len(unique_categories),
                    'variables_analyzed': numeric_vars
                }
            }
            
            logger.info("Categorical group analysis completed successfully")
            return result
            
        except Exception as e:
            logger.error(f"Error in categorical group analysis: {str(e)}")
            return f"Error in categorical group analysis: {str(e)}"

In [6]:
class MultivariateRAG:
    def __init__(self, api_key):
        """Initialize with OpenAI API key"""
        try:
            self.llm = OpenAI(openai_api_key=api_key, temperature=0)
            self.embeddings = OpenAIEmbeddings(openai_api_key=api_key)
            self.vector_store = None
            logger.info("MultivariateRAG initialized successfully")
        except Exception as e:
            logger.error(f"Error initializing MultivariateRAG: {str(e)}")
            raise
        
    def setup_rag(self, pdf_path):
        """Setup RAG system with PDF document"""
        try:
            if not os.path.exists(pdf_path):
                error_msg = f"Error: PDF file '{pdf_path}' not found"
                logger.error(error_msg)
                return error_msg
                
            loader = PyPDFLoader(pdf_path)
            documents = loader.load()
            
            text_splitter = CharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            texts = text_splitter.split_documents(documents)
            
            self.vector_store = FAISS.from_documents(texts, self.embeddings)
            logger.info("RAG system initialized successfully")
            return "RAG system initialized successfully"
            
        except Exception as e:
            error_msg = f"Error setting up RAG system: {str(e)}"
            logger.error(error_msg)
            return error_msg
        
    def create_tools(self, multivariate_analyzer):
        """Create tools for the agent"""
        try:
            tools = [
                Tool(
                    name="Multivariate Statistics",
                    func=lambda x: multivariate_analyzer.compute_multivariate_stats(x.split(',')),
                    description="Analyze relationships between multiple numeric variables. Input: comma-separated variable names"
                ),
                Tool(
                    name="PCA Analysis",
                    func=lambda x: multivariate_analyzer.perform_pca(x.split(',')),
                    description="Perform PCA to identify main patterns in numeric variables. Input: comma-separated variable names"
                ),
                Tool(
                    name="Categorical Group Analysis",
                    func=lambda x: multivariate_analyzer.analyze_by_categorical(
                        x.split(',')[:-1],
                        x.split(',')[-1]
                    ),
                    description="Analyze how relationships vary across groups. Input: numeric variables followed by categorical variable"
                ),
                Tool(
                    name="RAG Query",
                    func=lambda x: self.query_rag(x),
                    description="Query documentation for insights about multivariate patterns"
                )
            ]
            logger.info("Tools created successfully")
            return tools
        except Exception as e:
            logger.error(f"Error creating tools: {str(e)}")
            raise

    def setup_agent(self, tools):
        """Initialize LangChain agent"""
        try:
            agent = initialize_agent(
                tools,
                self.llm,
                agent="zero-shot-react-description",
                verbose=True,
                max_iterations=5
            )
            logger.info("Agent setup completed successfully")
            return agent
        except Exception as e:
            logger.error(f"Error setting up agent: {str(e)}")
            raise
        
    def query_rag(self, query):
        """Query the RAG system"""
        try:
            if self.vector_store is None:
                return "RAG system not initialized. Please load a PDF first."
                
            logger.info(f"Processing RAG query: {query}")
            relevant_docs = self.vector_store.similarity_search(query, k=3)
            context = "\n".join([doc.page_content for doc in relevant_docs])
            
            prompt = f"""
            Context: {context}
            
            Query: {query}
            
            Based on the context above and the insurance dataset, provide a detailed multivariate analysis focusing on:
            1. Key patterns and relationships
            2. Important interactions between variables
            3. Group-specific patterns if relevant
            4. Practical implications of the findings
            5. Potential actionable insights
            """
            
            response = self.llm(prompt)
            logger.info("RAG query processed successfully")
            return response
            
        except Exception as e:
            error_msg = f"Error querying RAG system: {str(e)}"
            logger.error(error_msg)
            return error_msg

In [7]:
def main():
    """Main function to run the entire analysis system"""
    try:
        # Load data
        print("\nInitializing Analysis System...")
        data = pd.read_csv('insurance.csv')
        logger.info("Data loaded successfully")
        print("Data loaded successfully")
        
        # Initialize analyzers
        multivariate_analyzer = MultivariateAnalyzer(data)
        api_key = os.getenv('OPENAI_API_KEY')
        if not api_key:
            raise ValueError("OpenAI API key not found in environment variables")
        rag_system = MultivariateRAG(api_key)
        logger.info("Analyzers initialized")
        print("Analyzers initialized")
        
        # Setup RAG system
        rag_status = rag_system.setup_rag('analysisRag.pdf')
        logger.info(f"RAG setup status: {rag_status}")
        print(f"✓ RAG setup status: {rag_status}")
        
        # Create and setup agent
        tools = rag_system.create_tools(multivariate_analyzer)
        agent = rag_system.setup_agent(tools)
        logger.info("Agent setup completed")
        print(" Agent setup completed")
        
        # Run the analyses
        print("\nRunning Multivariate Analyses...")
        run_multivariate_queries(multivariate_analyzer, rag_system, agent, data)
        
        # Generate additional insights
        print("\nGenerating Additional Insights...")
        run_additional_insights(rag_system)
        
        logger.info("Analysis completed successfully")
        print("\nAnalysis Complete!")
        
    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
        print(f"\nError in main execution: {str(e)}")
        print("\nPlease ensure:")
        print("1. 'insurance.csv' file is in the working directory")
        print("2. 'analysis_guidelines.pdf' file is in the working directory")
        print("3. OPENAI_API_KEY environment variable is set")
        print("\nCheck the log file for detailed error information.")

if __name__ == "__main__":
    main()

2024-12-17 10:53:24,437 - INFO - Data loaded successfully
2024-12-17 10:53:24,438 - INFO - MultivariateAnalyzer initialized successfully



Initializing Analysis System...
Data loaded successfully


2024-12-17 10:53:24,698 - INFO - MultivariateRAG initialized successfully
2024-12-17 10:53:24,698 - INFO - Analyzers initialized
2024-12-17 10:53:24,747 - WARNING - Ignoring wrong pointing object 6 0 (offset 0)
2024-12-17 10:53:24,748 - WARNING - Ignoring wrong pointing object 8 0 (offset 0)
2024-12-17 10:53:24,748 - WARNING - Ignoring wrong pointing object 10 0 (offset 0)
2024-12-17 10:53:24,749 - WARNING - Ignoring wrong pointing object 26 0 (offset 0)
2024-12-17 10:53:24,749 - WARNING - Ignoring wrong pointing object 28 0 (offset 0)


✓ Analyzers initialized


2024-12-17 10:53:25,351 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-17 10:53:25,402 - INFO - Loading faiss.
2024-12-17 10:53:25,490 - INFO - Successfully loaded faiss.
2024-12-17 10:53:25,494 - INFO - RAG system initialized successfully
2024-12-17 10:53:25,494 - INFO - RAG setup status: RAG system initialized successfully
2024-12-17 10:53:25,494 - INFO - Tools created successfully
2024-12-17 10:53:25,502 - INFO - Agent setup completed successfully
2024-12-17 10:53:25,502 - INFO - Agent setup completed
2024-12-17 10:53:25,503 - INFO - Processing query 1: Analyze the relationships between age, bmi, and charges


...


Insight Result:

1. Key patterns and relationships:
- Age and charges have a positive linear relationship, indicating that as people get older, they tend to pay more for insurance.
- BMI and charges do not have a clear relationship, suggesting that BMI may not have a significant impact on insurance costs.
- Age and BMI do not have a clear relationship, indicating that age may have a stronger influence on insurance costs than BMI.
- Smokers tend to pay higher insurance charges compared to non-smokers.
- There is a strong positive relationship between age and charges, indicating that as people age, their insurance costs increase.
- There is a weak positive relationship between BMI and age, charges and BMI, with some outliers.
- Smokers from all regions tend to pay higher insurance charges compared to non-smokers.

2. Important interactions between variables:
- The relationship between age and charges is influenced by smoking status, as smokers experience a bigger increase in insurance c

2024-12-17 10:54:14,375 - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-12-17 10:54:14,379 - INFO - RAG query processed successfully
2024-12-17 10:54:14,380 - INFO - Processing insight query 4: What patterns emerge from the multivariate analysis across different groups?
2024-12-17 10:54:14,381 - INFO - Processing RAG query: What patterns emerge from the multivariate analysis across different groups?



Insight Result:

1. Key patterns and relationships:
- Age and charges have a positive linear relationship, indicating that as people get older, they tend to pay more for insurance.
- BMI and charges do not have a clear relationship, suggesting that BMI may not have a significant impact on insurance charges.
- Age and BMI do not have a clear relationship, indicating that age may have a stronger influence on insurance charges than BMI.
- Smokers tend to pay higher insurance charges compared to non-smokers.
- There is a strong positive relationship between age and charges for smokers, indicating that as smokers age, their insurance charges increase significantly.
- The region where a person lives does not seem to have a significant impact on insurance charges, but there are some differences between regions for smokers and non-smokers.

2. Important interactions between variables:
- There is a strong positive relationship between age and charges for smokers, but this relationship is weake

2024-12-17 10:54:14,647 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-17 10:54:17,958 - INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-12-17 10:54:17,962 - INFO - RAG query processed successfully
2024-12-17 10:54:17,963 - INFO - Analysis completed successfully



Insight Result:

1. Key patterns and relationships:
- Age and charges have a positive linear relationship, indicating that as people get older, they tend to pay more for insurance.
- BMI and age have a weak positive relationship, suggesting that older individuals tend to have higher BMI.
- There is a weak or negligible relationship between BMI and charges, indicating that BMI does not have a significant impact on insurance charges.
- Smokers tend to pay higher insurance charges compared to non-smokers.
- There is a strong positive relationship between age and charges for smokers, indicating that as smokers get older, they experience a bigger increase in insurance charges compared to non-smokers.
- There is a weak positive relationship between BMI and charges for smokers, suggesting that smokers with higher BMI tend to pay slightly higher insurance charges.
- There is a weak positive relationship between BMI and age for smokers, indicating that smokers tend to have higher BMI as they g